In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from utils import load_galaxy_data

from visualize import visualize_activations

import app


input_data, labels = load_galaxy_data()

print(input_data.shape)
print(labels.shape)

input_data_train, input_data_valid, labels_train, labels_valid = train_test_split(input_data, labels, test_size=0.20, random_state=222, shuffle=True, stratify=labels)

data_generator = ImageDataGenerator(rescale=1./255)

training_iterator = data_generator.flow(input_data, labels,batch_size=5)
validation_iterator = data_generator.flow(input_data, labels,batch_size=5)

model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=(128,128,3)))



model.add(tf.keras.layers.Conv2D(8,3,strides=2,padding="valid",activation="relu"))

max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2))
model.add(max_pool_2d)

model.add(tf.keras.layers.Conv2D(8,3,strides=2,padding="valid",activation="relu"))

max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2))
model.add(max_pool_2d)

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(16,activation="relu"))

model.add(tf.keras.layers.Dense(4, activation='softmax'))

print(model.summary())

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.AUC()])

model.fit(training_iterator, steps_per_epoch=len(input_data_train)/5, epochs=8, validation_data=validation_iterator, validation_steps=len(input_data_valid)/5)

visualize_activations(model,validation_iterator)
